# Linear Regression Using Gradient Descent

Linear Regression is a supervised machine learning method to predict continuous values in data that follows a linear 
tendency.

For instance, the chart below shows data about houses prices (Y-axis) given their sizes (X-axis).
All prices are divided by 1,000 in the chart. The sizes are represented in square feet, they are divided by 1,000 too.
this data were collected in 1977, so the prices are much lower than the actual prices.

We call this data the training data. You can find them at this link: http://people.sc.fsu.edu/~jburkardt/datasets/regression/x27.txt

In [1]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
output_notebook()

sizes = [0.998, 1.500, 1.175, 1.232, 1.121, 0.988, 1.240, 1.501, 1.225, 1.552, 0.975, 1.121, 1.020,
        1.501, 1.664, 1.488, 1.376, 1.500, 1.256, 1.690, 1.820, 1.652, 1.777, 1.504, 1.831, 1.200 ]
prices = [25.9, 29.5, 27.9, 25.9, 29.9, 29.9, 30.9, 28.9, 35.9, 31.5, 31.0, 30.9, 30.0, 28.9, 36.9,
          41.9, 40.5, 43.9, 37.5, 37.9, 44.5, 37.9, 38.9, 36.9, 45.8, 41.0]

p = figure(plot_width=700, plot_height=250)
p.circle(sizes, prices)
show(p, notebook_handle=True)

Loading BokehJS ...

Using this data, how can we know what is the price of a house with size 1,400 square feet?

One way to answer this question is to draw a line following the linear trend of the training data. After, use the line as a guide and get the point that crosses the line with 1.4 value in the X-axis. Its Y-axis value is the prediction of the house price.

See the chart below.

In [2]:
p.line([1,2], [29.3015, 43.3078], line_width=2)
show(p, notebook_handle=True)

Visually we can notice that the price of a house with 1,400 square feet, according to the line, is close to 35,000.

Linear Regression can generate this line that follows the linear trend of bidimensional data. 

In math, a linear function defines a line: $f(x) = w_1x + w_0$, where $w_0$ and $w_1$ are the function coefficients. $w_1$ is the slope of the line and $w_0$ is the intercept, that is, the point where the line crosses the Y-axis.

In the chart above the linear function is $f(x) = 14.0063x + 15.2952$.

Linear regression finds a line the best fits the data. In other words, it finds a line such that the mean of the distances of the points to the line is the smallest.

The line function minimizes the equation: 

$$\frac{1}{2m} \sum_{i=1}^{m} (f(x_i) - y_i)^2$$

where $m$ is the number of points in the training data, and ($x_i$, $y_i$) defines a point.

In the summation, $(f(x_i) - y_i)$ is the distance between a $(x_i, y_i)$ point and the line, because $y_i$ is the point value in the Y-axis and $f(x_i)$ is the value estimated by the line.
The subtraction to the power of 2 ensures that the distances are always positive. So, it does not matter if the point is above or below the line.
In the end, the mean is calculated dividing the summation by $2m$. The number 2 will make some calculations in the following steps easier.

We can write the summation as a function of $w_0$ e $w_1$:

\begin{equation*}
J(w_0, w_1) = \frac{1}{2m} \sum_{i=1}^{m} (w_1x + w_0 - y_i)^2
\end{equation*}

This function is called the Mean Squared Error.

Let's define a Python function to calculate de Mean Squared Error:

In [3]:
def mean_squared_error(xs, ys, w0, w1):
    total = 0

    for i in range(len(xs)):
        x, y = xs[i], ys[i]
        fx = w1*x + w0
        error = (fx - y)**2
        total += error

    return total/(2*len(xs))

Calculating the Mean Squared Error of the line function showed before, where $w_0 = 15.2952$ e $w_1 = 14.0063$:

In [4]:
print("The Mean Squared Error is: {}".format(mean_squared_error(sizes, prices, 15.2952, 14.0063)))

The Mean Squared Error is: 10.553925770613349


To minimize $J(w_0, w_1)$ we are going to use the gradient descent algorithm.

The algorithm starts setting two random values for $w_0$ e $w_1$, and after that, step by step, it changes these values to minimize $J(w_0, w_1)$ gradually. The step size is defined by a parameter $\alpha$ called learning rate. If $\alpha$ is close to zero (for instance, 0.0001) the algorithm may take too long to converge. On the other hand, if $\alpha$ is a large value may the steps are too big in a way the algorithm never converges.

To discover which direction $w_0$ and $w_1$ should be changed to minimize $J(w_0, w_1)$, we will calculate the function gradient. For this, we need the partial derivatives of $J(w_0, w_1)$.

- $\frac{\partial}{\partial w_0} J(w_0, w_1) = \frac{1}{m} \sum_{i=1}^{m} (f(x_i) - y_i)$
- $\frac{\partial}{\partial w_1} J(w_0, w_1) = \frac{1}{m} \sum_{i=1}^{m} ((f(x_i) - y_i)x_i)$

The functions below in Python calculates the partial derivatives:

In [5]:
def d_w0(xs, ys, w0, w1):
    total = 0

    for i in range(len(xs)):
        x, y = xs[i], ys[i]
        fx = w1*x + w0
        error = (fx - y)
        total += error

    return total/(len(xs))


def d_w1(xs, ys, w0, w1):
    total = 0

    for i in range(len(xs)):
        x, y = xs[i], ys[i]
        fx = w1*x + w0
        error = (fx - y)
        total += error*x

    return total/(len(xs))

Knowing how to calculate the partial derivatives, we can define the algorithm to update $w_0$ and $w_1$.

Repeat until the values converge:

- $w_0 := w_0 - \alpha \frac{\partial}{\partial w_0} J(w_0, w_1)$
- $w_1 := w_1 - \alpha \frac{\partial}{\partial w_1} J(w_0, w_1)$

Note: the values of $w_0$ and $w_1$ must be updated simultaneously.

Below there is a code implementing the gradient descent algorithm. In each iteration, we store the values of $w_0$, $w_1$, and the mean squared error. Thus, we can analyze the evolution of our solution. In this implementation, the algorithm stops when the mean squared error does not decrease at least 0.00001 in an iteration.

In [6]:
import random

w0 = random.uniform(-2, 2)
w1 = random.uniform(-2, 2)
alpha = 0.1

coefficients = [(w0, w1)]
errors = [mean_squared_error(sizes, prices, w0, w1)]

while True:
    w0i = w0 - alpha*d_w0(sizes, prices, w0, w1)
    w1i = w1 - alpha*d_w1(sizes, prices, w0, w1)

    w0 = w0i
    w1 = w1i
    
    coefficients.append((w0, w1))
    errors.append(mean_squared_error(sizes, prices, w0, w1))
    
    if (errors[-2] - errors[-1] < 0.00001):
        break

print("Mean squared error: {}".format(mean_squared_error(sizes, prices, w0, w1)))
print("f(x) = {}x + {}".format(w1, w0))

Mean squared error: 10.556043392626796
f(x) = 14.251517157241972x + 14.948426103202987


The following chart shows the function found:

In [7]:
p_final = figure(plot_width=700, plot_height=250)
p_final.circle(sizes, prices)
p_final.line([1,2], [w1 + w0, w1*2 + w0], line_width=2)
show(p_final, notebook_handle=True)

Using the stored values of $w_0$ and $w_1$ let's see the evolution of the function during the iterations of the gradient descent. 

The code below shows the function found in the first 12 iterations:

In [8]:
from bokeh.layouts import gridplot

plots_grid = [[]]
for i in range(12):
    w0, w1 = coefficients[i]
    
    p = figure(plot_width=250, plot_height=200)
    p.circle(sizes, prices)
    p.line([1,2], [w1 + w0, w1*2 + w0], line_width=2)
    
    if (len(plots_grid[-1]) == 3):
        plots_grid.append([])
    
    plots_grid[-1].append(p)
    
p = gridplot(plots_grid)
show(p)

After a few iterations, the function is very similar to our final result.

Another way to analyze the evolution of our solution is plotting the mean squared error values for each iteration.

In [9]:
error_points = [[], []]
for i, error in enumerate(errors):
    error_points[0].append(i)
    error_points[1].append(error)

error_chart = figure(plot_width=800, plot_height=300)
error_chart.line(error_points[0], error_points[1], line_width=2)
show(error_chart)

The mean squared error falls quickly in the beginning. This confirms what we saw before, that the lines in the first iterations are almost equal to the final solution.